In [1]:
# =========================
# 树莓派彩色目标追踪 + 保存画面（Jupyter Notebook 完整版）
# =========================

from Raspblock import Raspblock
import cv2
import numpy as np
import threading
import time
import inspect
import ctypes
import traitlets
import ipywidgets as widgets
from IPython.display import display
import PID

# =========================
# 初始化机器人与摄像头
# =========================
robot = Raspblock()

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

image_widget = widgets.Image(format='jpeg', width=300, height=300)
display(image_widget)

image = cv2.VideoCapture(0)
image.set(3, 320)
image.set(4, 240)
image.set(5, 120)  # 设置帧率
image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M','J','P','G'))

ret, frame = image.read()
if ret:
    image_widget.value = bgr8_to_jpeg(frame)

# =========================
# 全局变量初始化
# =========================
color_x = color_y = color_radius = 0
target_valuex = target_valuey = 1500
g_mode = 0

color_lower = np.array([156, 43, 46])
color_upper = np.array([180, 255, 255])

xservo_pid = PID.PositionalPID(1.1, 0.2, 0.8)
yservo_pid = PID.PositionalPID(0.8, 0.2, 0.8)

current_frame = None  # 保存最新一帧，用于保存画面

# =========================
# 停止线程函数
# =========================
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)

def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# =========================
# 按钮定义
# =========================
Redbutton = widgets.Button(description='红色', icon='uncheck')
Greenbutton = widgets.Button(description='绿色', icon='uncheck')
Bluebutton = widgets.Button(description='蓝色', icon='uncheck')
Yellowbutton = widgets.Button(description='黄色', icon='uncheck')
Orangebutton = widgets.Button(description='橙色', icon='uncheck')
Closebutton = widgets.Button(description='关闭', icon='uncheck')
Savebutton = widgets.Button(description='保存画面', icon='save')
output = widgets.Output()

# 一行显示所有按钮
buttons_box = widgets.HBox([Redbutton, Greenbutton, Bluebutton, Yellowbutton, Orangebutton, Closebutton, Savebutton])
display(buttons_box, output)

# =========================
# 按钮回调函数
# =========================
def ALL_Uncheck():
    for btn in [Redbutton, Greenbutton, Bluebutton, Yellowbutton, Orangebutton]:
        btn.icon = 'uncheck'

def on_Redbutton_clicked(b):
    global color_lower, color_upper, g_mode, target_valuex, target_valuey
    ALL_Uncheck(); b.icon='check'
    color_lower = np.array([0, 43, 46])
    color_upper = np.array([10, 255, 255])
    target_valuex = target_valuey = 2048
    robot.Servo_control(1000,1500)
    g_mode = 1
    with output: print("红色模式启用")

def on_Greenbutton_clicked(b):
    global color_lower, color_upper, g_mode, target_valuex, target_valuey
    ALL_Uncheck(); b.icon='check'
    color_lower = np.array([35, 43, 46])
    color_upper = np.array([77, 255, 255])
    target_valuex = target_valuey = 2048
    robot.Servo_control(1500,1500)
    g_mode = 1
    with output: print("绿色模式启用")

def on_Bluebutton_clicked(b):
    global color_lower, color_upper, g_mode, target_valuex, target_valuey
    ALL_Uncheck(); b.icon='check'
    color_lower = np.array([100,43,46])
    color_upper = np.array([124,255,255])
    target_valuex = target_valuey = 2048
    robot.Servo_control(1500,1500)
    g_mode = 1
    with output: print("蓝色模式启用")

def on_Yellowbutton_clicked(b):
    global color_lower, color_upper, g_mode, target_valuex, target_valuey
    ALL_Uncheck(); b.icon='check'
    color_lower = np.array([26,43,46])
    color_upper = np.array([34,255,255])
    target_valuex = target_valuey = 1500
    robot.Servo_control(1500,1500)
    g_mode = 1
    with output: print("黄色模式启用")

def on_Orangebutton_clicked(b):
    global color_lower, color_upper, g_mode, target_valuex, target_valuey
    ALL_Uncheck(); b.icon='check'
    color_lower = np.array([11,43,46])
    color_upper = np.array([25,255,255])
    target_valuex = target_valuey = 2048
    robot.Servo_control(1500,1500)
    g_mode = 1
    with output: print("橙色模式启用")

def on_Closebutton_clicked(b):
    global g_mode
    ALL_Uncheck()
    g_mode = 0
    with output: print("追踪停止")

def on_Savebutton_clicked(b):
    global current_frame
    if current_frame is not None:
        filename = f"capture_{int(time.time())}.jpg"
        cv2.imwrite(filename, current_frame)
        with output: print(f"✅ 已保存画面到 {filename}")
    else:
        with output: print("⚠ 当前没有画面可以保存")

# =========================
# 绑定按钮事件
# =========================
Redbutton.on_click(on_Redbutton_clicked)
Greenbutton.on_click(on_Greenbutton_clicked)
Bluebutton.on_click(on_Bluebutton_clicked)
Yellowbutton.on_click(on_Yellowbutton_clicked)
Orangebutton.on_click(on_Orangebutton_clicked)
Closebutton.on_click(on_Closebutton_clicked)
Savebutton.on_click(on_Savebutton_clicked)

# =========================
# 彩色目标追踪线程
# =========================
def Color_track():
    global color_lower, color_upper, g_mode, target_valuex, target_valuey, current_frame
    t_start = time.time()
    fps = 0
    while True:
        ret, frame = image.read()
        frame = cv2.resize(frame,(300,300))
        frame_ = cv2.GaussianBlur(frame,(5,5),0)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv,color_lower,color_upper)
        mask = cv2.erode(mask,None,iterations=2)
        mask = cv2.dilate(mask,None,iterations=2)
        mask = cv2.GaussianBlur(mask,(3,3),0)
        cnts = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]

        if g_mode==1:
            if len(cnts)>0:
                cnt = max(cnts,key=cv2.contourArea)
                (color_x,color_y),color_radius = cv2.minEnclosingCircle(cnt)
                if color_radius>10:
                    cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)
                    # X轴PID
                    xservo_pid.SystemOutput = color_x
                    xservo_pid.SetStepSignal(150)
                    xservo_pid.SetInertiaTime(0.01,0.1)
                    target_valuex = int(1500 + xservo_pid.SystemOutput)
                    # Y轴PID
                    yservo_pid.SystemOutput = color_y
                    yservo_pid.SetStepSignal(150)
                    yservo_pid.SetInertiaTime(0.01,0.1)
                    target_valuey = int(1500 - yservo_pid.SystemOutput)
                    # 云台控制
                    robot.Servo_control(target_valuex,target_valuey)

        fps += 1
        mfps = fps / (time.time() - t_start)
        cv2.putText(frame, f"FPS {int(mfps)}",(40,40),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),3)

        # 更新实时画面
        image_widget.value = bgr8_to_jpeg(frame)
        current_frame = frame.copy()  # 保存当前帧

# =========================
# 启动追踪线程
# =========================
thread1 = threading.Thread(target=Color_track)
thread1.setDaemon(True)
thread1.start()


serial Open!


Image(value=b'', format='jpeg', height='300', width='300')

Output()